In [48]:
import pandas as pd
import seaborn as sns
import random
import numpy as np
iris = pd.read_csv('iris.csv')

class ProcessingData:
    
    @staticmethod
    def shuffle(base):
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalize(base):
        base.iloc[:,:-1] = (base.iloc[:,:-1]-base.iloc[:,:-1].min())/(base.iloc[:,:-1].max()-base.iloc[:,:-1].min())

        
    @staticmethod
    def devide(base):
        n = int(len(base)*70/100)
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

class KNN:

    def distance(x, y, m):
        dist = 0
        for i in range(len(y)-1):
            dist += abs(x[i]-y[i])**m
        dist = float(dist**(1/m))
        return dist

    @staticmethod
    def sorting(distances, trainList):
        trainBase = trainList.copy()
        for i in range(len(trainBase)):
            ready = True
            for j in range(len(trainBase)-i-1):
                if distances[j]>distances[j+1]:
                    distances[j], distances[j+1] = distances[j+1], distances[j]
                    trainBase.iloc[j], trainBase.iloc[j+1] = trainBase.iloc[j+1], trainBase.iloc[j]
                    ready = False
            if ready:
                break
        return distances, trainBase

    @staticmethod
    def clustering(sample, data, k, m):
        corrected = 0 #ile mądry komputer zgadną poprawnie
        n = len(sample)  #Długość bazy testowej
        for irisNum in range(n):
            testSample = sample.iloc[irisNum] #Biorę jeden kwiatek z walidacyjnego 
            realIris = sample.iloc[irisNum].variety #Zapisuję jego prawdziwą nazwę do zmiennej
            classes = {"Setosa": 0, "Virginica": 0, "Versicolor": 0}
            
            distances = [] # lista odległości do wszystkich kwiatków w zbiorze testowym
            for i in range(len(data)): #dla każdej wartości w zbiorze testowym
                distances.append(KNN.distance(testSample,data.iloc[i],m)) #obliczamy odległość dla danego kwiatka z bazy testowej
            
            distances, data = KNN.sorting(distances, data) #Sortowanie

            for i in range(0,k,1): #Biorę tyle kwiatków z najmniejszą odległością ile wynosi k
                classes[data.iloc[i].variety] +=1 #dodaję jeden dla nazwy która ma najmniejszą odległość
            
            winnerIris = max(classes,key=classes.get) # wyzancz zwycięzce w głosowaniu "Jaki to kwiatek"
            if winnerIris == realIris: 
                corrected +=1 #Jeżeli zgadł to dodaj 1 do poprawnie odgadniętych

        accuracy = corrected / len(sample)*100

        return accuracy
    
        
irisEdit = iris.copy()

ProcessingData.shuffle(irisEdit)
ProcessingData.normalize(irisEdit)
trening_data,validate_data = ProcessingData.devide(irisEdit)

In [49]:
m = 2
for k in range (2,5):
    print("k={} m={} => Dokładność - {:.2f}%".format(k,m,KNN.clustering(validate_data,trening_data,k,m)))

k=2 m=2 => Dokładność - 95.56%
k=3 m=2 => Dokładność - 100.00%
k=4 m=2 => Dokładność - 100.00%
